GET A MPS WITH NATIVE BOND DIMENSION 2

In [3]:
import numpy as np
from ncon import ncon
from numpy import linalg as LA 
from scipy.linalg import null_space as NS 

In [8]:
N = 3
mps = np.random.random(2**N) + np.random.random(2**N) * 1j
mps_norm = mps / LA.norm(mps)
psi_norm = mps_norm / LA.norm(mps_norm)
psi_norm_tens = psi_norm.reshape(2,2,2)
print(LA.norm(psi_norm))
print(psi_norm)

1.0
[0.15780411+0.27784967j 0.24836364+0.1922677j  0.21830191+0.05562632j
 0.32103379+0.08629133j 0.39216154+0.0204695j  0.37886592+0.17955423j
 0.08499291+0.39532217j 0.05243609+0.37649091j]


In [9]:
mps_sites = [0 for _ in range(N)]


Atemp, Stemp, Vhtemp = LA.svd(mps_norm.reshape(2,2**(N-1)), full_matrices=False)
mps_sites[0] = Atemp.reshape(1,2,2).transpose(0,2,1)  #shape della matrice A_1 ---> A_8 paper è 2,2 - mapping 1 a 1
mps_sites[1] = (np.diag(Stemp)@Vhtemp)/LA.norm(Stemp)

for i in range(1,N-1):
    Atemp, Stemp, Vhtemp = LA.svd(mps_sites[i].reshape(2**2,2**(N-i-1)), full_matrices=False)
    mps_sites[i] = Atemp[:,:2].reshape(2,2,2)
    mps_sites[i+1] = ((np.diag(Stemp[:2])@Vhtemp[:2,:]).reshape(2,-1))/LA.norm(Stemp)

mps_sites[-1] = ((np.diag(Stemp[:2])@Vhtemp[:2,:]).reshape(2,2,1))/LA.norm(Stemp)

mps_2 = ncon([mps_sites[0], mps_sites[1], mps_sites[2]],[[-1,1,-2],[1,2,-3],[2,-4,-5]])
print("Error from MPS to MPS(2):\n",LA.norm(mps_norm.reshape(2,2,2) - mps_2[0,:,:,0,:]) / LA.norm(mps_norm))



Error from MPS to MPS(2):
 1.417198844709443


In [12]:
print(np.round(ncon([mps_sites[0].conj(), mps_sites[0]],[[1,2,-1],[1,2,-2]])))

print(mps_sites[0].shape)
k_a1 = NS(mps_sites[0].reshape(2,2).conj().T)
print(mps_sites[1].shape)
k_a2 = NS(mps_sites[1].reshape(4,2).conj().T)
print(mps_sites[2].shape)
k_a3 = NS(mps_sites[2].reshape(4,1).conj().T)
print(k_a3)

[[1.-0.j 0.-0.j]
 [0.+0.j 1.+0.j]]
(1, 2, 2)
(2, 2, 2)
(2, 2, 1)
[[-0.71752729+5.49725692e-02j -0.11442899-0.00000000e+00j
   0.10723536-8.21571996e-03j]
 [ 0.69107746+1.86083560e-18j -0.0489785 -3.75243441e-03j
   0.04616886+2.59102316e-19j]
 [-0.0489785 +3.75243441e-03j  0.99218906-0.00000000e+00j
   0.0073199 -5.60806066e-04j]
 [ 0.04616886-2.07360633e-20j  0.0073199 +5.60806066e-04j
   0.9931    -7.80720333e-20j]]


In [31]:
for k in null_spaces:
    print("----")
    print("Null space:\n",k)
    print("Shape:\n",k.shape)
    print("Identity:\n",k.conj().T@k)

----
Null space:
 []
Shape:
 (2, 0)
Identity:
 []
----
Null space:
 [[0. 0.]
 [0. 1.]
 [1. 0.]
 [0. 0.]]
Shape:
 (4, 2)
Identity:
 [[1. 0.]
 [0. 1.]]
----
Null space:
 [[0. 0.]
 [0. 1.]
 [1. 0.]
 [0. 0.]]
Shape:
 (4, 2)
Identity:
 [[1. 0.]
 [0. 1.]]
